In [6]:
import sqlite3
import pandas as pd
import numpy as np
import sklearn
import surprise

In [7]:
con = sqlite3.connect("db.sqlite3")
cur = con.cursor()

In [8]:
df_store = pd.read_sql_query("SELECT * FROM api_store", con)
df_review = pd.read_sql_query("SELECT * FROM api_review", con)

In [9]:
df_store.head()

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차,0,0
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페,0,0
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍,0,0
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제,0,0
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트,0,0


In [10]:
df_review.head()

,id,total_score,content,reg_time,store_id,user_id
0,1,5.0,전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!,2020-04-07 07:13:49.444324,15,68632
1,2,5.0,샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천,2020-04-07 07:13:49.444324,18,389728
2,3,4.0,홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...,2020-04-07 07:13:49.444324,19,68716
3,4,2.0,"전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건...",2020-04-07 07:13:49.444324,37,774353
4,5,3.0,친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...,2020-04-07 07:13:49.444324,38,115682


In [11]:
con.close()

In [12]:
stores = df_store[['id', 'store_name', 'category']]
reviews = df_review[['user_id', 'store_id', 'total_score']]
stores.rename(columns={'id': 'store_id'}, inplace=True)
reviews.rename(columns={'total_score': 'rating'}, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
df = pd.merge(reviews, stores, on="store_id")
df.head()

,user_id,store_id,rating,store_name,category
0,68632,15,5.0,써리힐,호주레스토랑
1,389728,18,5.0,진삼미 샌드위치,샌드위치|쥬스
2,68716,19,4.0,한옥마을 전주비빔밥,전주비빔밥
3,774353,37,2.0,007식당,굴비정식|굴비
4,115682,38,3.0,010수산,횟집


In [14]:
df.shape

(91398, 5)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91398 entries, 0 to 91397
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user_id     91398 non-null  int64  
 1   store_id    91398 non-null  int64  
 2   rating      91398 non-null  float64
 3   store_name  91398 non-null  object 
 4   category    91398 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 4.2+ MB


## Ratings Distribution

In [16]:
!pip install plotly

In [31]:
reader = surprise.Reader(rating_scale=(0, 5))
dataframe = surprise.Dataset.load_from_df(df[['user_id', 'store_id', 'rating']], reader)
trainset = dataframe.build_full_trainset()
global_mean = trainset.global_mean
bu = np.zeros(trainset.n_users)
bi = np.zeros(trainset.n_items)

In [32]:
reg = 0.02 # reg: 정규화 가중치
learning_rate = 0.005 # learning_rate: 최적화를 위한 스탭 사이즈
n_epochs = 20 # n_epochs: 최적화 반복 횟수

def bsl_option_is_sgd():
    for dummy in range(n_epochs):
        for u, i, r in self.trainset.all_ratings():
            err = (r - (global_mean + bu[u] + bi[i]))
            bu[u] += lr * (err - reg * bu[u])
            bi[i] += lr * (err - reg * bi[i])
    return bu, bi

In [33]:
reg_i = 10 # reg_i: 상품에 대한 가중치
reg_u = 15 # reg_u: 유저에 대한 가중치
n_epochs = 10 # n_epochs: 최적화 반복 횟수

def bsl_option_is_als():
    for dummy in range(n_epoch):
        for i in trainset.all_items():
            dev_i = 0
            for (u, r) in trainset.ir[i]:
                dev_i += r - global_mean - bu[u]

            bi[i] = dev_i / (reg_i + len(trainset.ir[i]))

        for u in trainset.all_users():
            dev_u = 0
            for (i, r) in trainset.ur[u]:
                dev_u += r - global_mean - bi[i]

            bu[u] = dev_u / (reg_u + len(trainset.ur[u]))
    return bu, bi

In [43]:
sim_options = {
    'name': 'pearson', # 사용할 유사도 이름. ['cosine', 'pearson_baseline', 'pearson']
    'user_based': True, # user를 base로 할것인지 item을 base로 할 것인지 결정.
    # u, v간 교집합 원소 최소 갯수를 설정한다. 이 경우는 5개 이상의 공통된 아이템(or 사용자)가
    # 존재하면 유사도를 계산하지만, 그 미만의 경우에는 sim(u, v) == 0로 설정한다.
    'min_support': 5,
    'shrinkage': 0, # 이 패러미터는 pearson_baseline을 사용할 경우에만 해당된다.
}
algo = surprise.KNNBasic(sim_options=sim_options)

In [47]:
from surprise.model_selection import cross_validate

In [48]:
cross_validate(algo, data)["test_mae"].mean()

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


0.8527115999606029

In [49]:
ratings = df
ratings.head()

,user_id,store_id,rating,store_name,category
0,68632,15,5.0,써리힐,호주레스토랑
1,389728,18,5.0,진삼미 샌드위치,샌드위치|쥬스
2,68716,19,4.0,한옥마을 전주비빔밥,전주비빔밥
3,774353,37,2.0,007식당,굴비정식|굴비
4,115682,38,3.0,010수산,횟집


In [50]:
from surprise import Reader, Dataset

In [52]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['user_id', 'store_id', 'rating']], reader)

In [53]:
# Split data into 5 folds

data.split(n_folds=5)

from surprise import SVD, evaluate
from surprise import NMF
from surprise import KNNBasic

# svd
algo = SVD()
evaluate(algo, data, measures=['RMSE'])

# nmf
algo = NMF()
evaluate(algo, data, measures=['RMSE'])

algo = KNNBasic()
evaluate(algo, data, measures=['RMSE'])

AttributeError: 'DatasetAutoFolds' object has no attribute 'split'

In [54]:
import pandas as pd

In [59]:
df_store.head()

,id,store_name,branch,area,tel,address,latitude,longitude,category,review_count,review_total_score
0,1,Agal,None,홍대,010-6689-5886,서울특별시 마포구 동교동 170-13,37.556862,126.926666,아구찜|포장마차,0,0
1,2,Assisy,None,광주,062-367-0700,광주광역시 서구 농성동 631-33,35.150746,126.890062,카페,0,0
2,3,Battered Sole,None,이태원,02-749-6867,서울특별시 용산구 이태원동 118-9,37.535032,126.991664,피쉬앤칩스|펍,0,0
3,4,Chakyoung,None,달맞이길,051-756-5566,부산광역시 해운대구 중2동 1509-5,35.158587,129.175004,레스토랑|카프레제,0,0
4,5,Delabobo,None,발산역,02-2667-9854,서울특별시 강서구 등촌동 689,37.559904,126.840512,디저트카페|디저트,0,0


In [58]:
df_review.head()

,id,total_score,content,reg_time,store_id,user_id
0,1,5.0,전포 윗길에 새로 생긴! 호주에서 온 쉐프가 직접 요리하는 호주식 레스토랑!,2020-04-07 07:13:49.444324,15,68632
1,2,5.0,샌드위치 내용물도 알차게 들어있고 맛있었어요 가성비 추천,2020-04-07 07:13:49.444324,18,389728
2,3,4.0,홈플러스 1층 매장 푸드코트 내 있는 매장인데 계란찜정식은 처음보고 시켜봣는데 사진...,2020-04-07 07:13:49.444324,19,68716
3,4,2.0,"전 여기 5년전에 가봤었는데 그때 기억은 별로였어요\n단체손님이 많았고, 차려지는건...",2020-04-07 07:13:49.444324,37,774353
4,5,3.0,친구들끼리 술 간단하게마시러 감. 스끼다시 괜찮지만 회 양이 조금 부족한 느낌. 맛...,2020-04-07 07:13:49.444324,38,115682


In [68]:
stores = df_store[['id', 'store_name', 'category']]
reviews = df_review[['user_id', 'store_id', 'total_score']]
stores.rename(columns={'id': 'store_id'}, inplace=True)
reviews.rename(columns={'total_score': 'rating'}, inplace=True)

In [69]:
Ratings = reviews

In [70]:
Ratings.head()

,user_id,store_id,rating
0,68632,15,5.0
1,389728,18,5.0
2,68716,19,4.0
3,774353,37,2.0
4,115682,38,3.0


In [72]:
Mean = Ratings.groupby(by="user_id", as_index=False)['rating'].mean()
Rating_avg = pd.merge(Ratings, Mean, on='user_id')
Rating_avg.head()

,user_id,store_id,rating_x,rating_y
0,68632,15,5.0,4.842975
1,68632,1216,5.0,4.842975
2,68632,8756,5.0,4.842975
3,68632,9460,5.0,4.842975
4,68632,13745,5.0,4.842975


In [ ]:
Rating_avg['adg_rating'] = Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()